In [1]:
pip install mysql-connector-python

   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB 435.7 kB/s eta 0:00:34
   ---------------------------------------- 0.0/14.5 MB 393.8 kB/s eta 0:00:37
   ---------------------------------------- 0.1/14.5 MB 435.7 kB/s eta 0:00:34
   ---------------------------------------- 0.1/14.5 MB 476.3 kB/s eta 0:00:31
   ---------------------------------------- 0.1/14.5 MB 479.3 kB/s eta 0:00:30
   ---------------------------------------- 0.1/14.5 MB 500.5 kB/s eta 0:00:29
   ---------------------------------------- 0.2/14.5 MB 499.5 kB/s eta 0:00:29
   ---------------------------------------- 0.2/14.5 MB 499.5 kB/s eta 0:00:29
   ---------------------------------------- 0.2/14.5 MB 499.5 kB/s eta 0:00:29
   ---------------------------------------- 0.2/14.5 MB 499.5 kB/s eta 0:00:29
   ---------------------------------------- 0.2/14.5 MB 499.5 kB/s eta

In [8]:
from tkinter import *
from tkinter.messagebox import showinfo
from tkinter.filedialog import askopenfilename, asksaveasfilename
from tkinter.simpledialog import askstring
import mysql.connector
import os

# MySQL connection setup
db_config = {
    'user': 'root',
    'password': '',
    'host': 'localhost',
    'database': 'notepad_db'
}

def get_db_connection():
    return mysql.connector.connect(**db_config)

# Function to create a new file
def newFile():
    global file
    root.title("Untitled - Notepad")
    file = None
    TextArea.delete(1.0, END)

# Function to open an existing file
def openFile():
    global file
    file = askopenfilename(defaultextension=".txt",
                           filetypes=[("All Files", "*.*"),
                                      ("Text Documents", "*.txt")])
    if file == "":
        file = None
    else:
        root.title(os.path.basename(file) + " - Notepad")
        TextArea.delete(1.0, END)
        with open(file, "r") as f:
            TextArea.insert(1.0, f.read())

# Function to save the current file
def saveFile():
    global file
    if file is None:
        file = asksaveasfilename(initialfile='Untitled.txt', defaultextension=".txt",
                                 filetypes=[("All Files", "*.*"),
                                            ("Text Documents", "*.txt")])
        if file == "":
            file = None
        else:
            # Save as a new file
            with open(file, "w") as f:
                f.write(TextArea.get(1.0, END))
            root.title(os.path.basename(file) + " - Notepad")
    else:
        # Save the file
        with open(file, "w") as f:
            f.write(TextArea.get(1.0, END))

# Function to save file to MySQL database
def saveToDatabase():
    filename = askstring("Save As", "Enter filename:")
    if filename:
        content = TextArea.get(1.0, END)
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute("INSERT INTO files (filename, content) VALUES (%s, %s)", (filename, content))
        conn.commit()
        cursor.close()
        conn.close()
        showinfo("Save to Database", "File saved to database successfully")

# Function to open file from MySQL database
def openFromDatabase():
    filename = askstring("Open", "Enter filename:")
    if filename:
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute("SELECT content FROM files WHERE filename=%s", (filename,))
        result = cursor.fetchone()
        cursor.close()
        conn.close()
        if result:
            TextArea.delete(1.0, END)
            TextArea.insert(1.0, result[0])
            root.title(filename + " - Notepad")
        else:
            showinfo("Open from Database", "File not found in database")

# Function to exit the application
def quitApp():
    root.destroy()

# Functions for cut, copy and paste functionalities
def cut():
    TextArea.event_generate(("<<Cut>>"))

def copy():
    TextArea.event_generate(("<<Copy>>"))

def paste():
    TextArea.event_generate(("<<Paste>>"))

# Function to show About info
def about():
    showinfo("Notepad", "Notepad 1.0")

# Function to search text
def find():
    def find_text():
        TextArea.tag_remove('found', '1.0', END)
        start_pos = '1.0'
        key = find_input.get()
        if key:
            while True:
                start_pos = TextArea.search(key, start_pos, stopindex=END)
                if not start_pos:
                    break
                end_pos = f'{start_pos}+{len(key)}c'
                TextArea.tag_add('found', start_pos, end_pos)
                start_pos = end_pos
            TextArea.tag_config('found', foreground='white', background='blue')
    
    find_popup = Toplevel(root)
    find_popup.title("Find")
    find_popup.geometry("300x100")
    find_label = Label(find_popup, text="Find")
    find_label.pack(side=LEFT, padx=10)
    find_input = Entry(find_popup, width=25)
    find_input.pack(side=LEFT, padx=10)
    find_button = Button(find_popup, text="Find", command=find_text)
    find_button.pack(side=LEFT, padx=10)

# Function to replace text
def replace():
    def replace_text():
        key = find_input.get()
        replacement = replace_input.get()
        content = TextArea.get(1.0, END)
        new_content = content.replace(key, replacement)
        TextArea.delete(1.0, END)
        TextArea.insert(1.0, new_content)
    
    replace_popup = Toplevel(root)
    replace_popup.title("Replace")
    replace_popup.geometry("400x150")
    find_label = Label(replace_popup, text="Find")
    find_label.pack()
    find_input = Entry(replace_popup, width=25)
    find_input.pack()
    replace_label = Label(replace_popup, text="Replace")
    replace_label.pack()
    replace_input = Entry(replace_popup, width=25)
    replace_input.pack()
    replace_button = Button(replace_popup, text="Replace", command=replace_text)
    replace_button.pack()

# Function to change font
def change_font():
    def set_font():
        font_family = font_input.get()
        font_size = size_input.get()
        TextArea.config(font=(font_family, font_size))
    
    font_popup = Toplevel(root)
    font_popup.title("Font")
    font_popup.geometry("400x150")
    font_label = Label(font_popup, text="Font")
    font_label.pack()
    font_input = Entry(font_popup, width=25)
    font_input.pack()
    size_label = Label(font_popup, text="Size")
    size_label.pack()
    size_input = Entry(font_popup, width=25)
    size_input.pack()
    font_button = Button(font_popup, text="Set Font", command=set_font)
    font_button.pack()

# Function to show word count
def word_count():
    content = TextArea.get(1.0, END)
    words = len(content.split())
    showinfo("Word Count", f"Words: {words}")

# Function to change theme
def change_theme(bg_color, fg_color):
    TextArea.config(bg=bg_color, fg=fg_color)
    
# Function to zoom in
def zoom_in():
    current_font = TextArea.cget("font")
    font_family, font_size = current_font.split()
    font_size = int(font_size) + 2
    TextArea.config(font=(font_family, font_size))

# Function to zoom out
def zoom_out():
    current_font = TextArea.cget("font")
    font_family, font_size = current_font.split()
    font_size = int(font_size) - 2
    TextArea.config(font=(font_family, font_size))

# Function to restore default zoom
def restore_default_zoom():
    TextArea.config(font=("lucida", 13))

if __name__ == '__main__':
    # Basic tkinter setup
    root = Tk()
    root.title("Untitled - Notepad")
    root.wm_iconbitmap("1.ico")
    root.geometry("800x600")

    # Add TextArea
    TextArea = Text(root, font="lucida 13")
    file = None
    TextArea.pack(expand=True, fill=BOTH)

    # Create a MenuBar
    MenuBar = Menu(root)

    # File Menu Starts
    FileMenu = Menu(MenuBar, tearoff=0)
    FileMenu.add_command(label="New", command=newFile)  # New file
    #FileMenu.add_command(label="Open", command=openFile)  # Open existing file
    #FileMenu.add_command(label="Save", command=saveFile)  # Save current file
    FileMenu.add_command(label="Open", command=openFromDatabase)  # Open from database
    FileMenu.add_command(label="Save", command=saveToDatabase)  # Save to database
    FileMenu.add_separator()
    FileMenu.add_command(label="Exit", command=quitApp)  # Exit application
    MenuBar.add_cascade(label="File", menu=FileMenu)

    # Edit Menu Starts
    EditMenu = Menu(MenuBar, tearoff=0)
    EditMenu.add_command(label="Cut", command=cut)  # Cut text
    EditMenu.add_command(label="Copy", command=copy)  # Copy text
    EditMenu.add_command(label="Paste", command=paste)  # Paste text
    EditMenu.add_command(label="Find", command=find)  # Find text
    EditMenu.add_command(label="Replace", command=replace)  # Replace text
    MenuBar.add_cascade(label="Edit", menu=EditMenu)

    # Format Menu Starts
    FormatMenu = Menu(MenuBar, tearoff=0)
    FormatMenu.add_command(label="Font", command=change_font)  # Change font
    MenuBar.add_cascade(label="Format", menu=FormatMenu)

   # View Menu Starts
    ViewMenu = Menu(MenuBar, tearoff=0)
    ViewMenu.add_command(label="Word Count", command=word_count)  # Word count
    ViewMenu.add_command(label="Zoom In", command=zoom_in)  # Zoom in
    ViewMenu.add_command(label="Zoom Out", command=zoom_out)  # Zoom out
    ViewMenu.add_command(label="Restore Default Zoom", command=restore_default_zoom)  # Restore default zoom
    MenuBar.add_cascade(label="View", menu=ViewMenu)

    # Theme Menu Starts
    ThemeMenu = Menu(MenuBar, tearoff=0)
    ThemeMenu.add_command(label="Light Theme", command=lambda: change_theme("white", "black"))  # Light theme
    ThemeMenu.add_command(label="Dark Theme", command=lambda: change_theme("black", "white"))  # Dark theme
    MenuBar.add_cascade(label="Theme", menu=ThemeMenu)

    # Help Menu Starts
    HelpMenu = Menu(MenuBar, tearoff=0)
    HelpMenu.add_command(label="About Notepad", command=about)  # About Notepad
    MenuBar.add_cascade(label="Help", menu=HelpMenu)

    # Configure the menu
    root.config(menu=MenuBar)

    # Adding Scrollbar
    Scroll = Scrollbar(TextArea)
    Scroll.pack(side=RIGHT, fill=Y)
    Scroll.config(command=TextArea.yview)
    TextArea.config(yscrollcommand=Scroll.set)

    # Adding Status Bar
    status_var = StringVar()
    status_var.set("Status Bar")
    status_bar = Label(root, textvariable=status_var, relief=SUNKEN, anchor=W)
    status_bar.pack(side=BOTTOM, fill=X)

    def update_status_bar(event=None):
        row, col = TextArea.index(INSERT).split('.')
        status_var.set(f"Row: {row}, Column: {col}")

    TextArea.bind("<KeyRelease>", update_status_bar)

    root.mainloop()
